In [3]:

import csv
import math 
import random
import gzip
import torch
from sklearn import metrics
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import gensim
import multiprocessing
import numpy as np
from torch.utils.data import Dataset, DataLoader
# from extract_motifs import get_motif
import torch.nn as nn
import os
import argparse
import warnings

#%%
from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)

In [4]:
train_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/gene_seq_train.csv')
train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/res_train.csv')
#don't touch test data, split out validation data from training data during training
test_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/gene_seq_test.csv')
test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/res_test.csv')

In [42]:
class Chip():
    def __init__(self,filename,motiflen=24):
        self.file = filename
        self.motiflen = motiflen
  
    def openFile(self):
        train_dataset=[]
        sequences=[]
        with gzip.open(self.file, 'rt') as data:
                next(data)
                reader = csv.reader(data,delimiter='\t')

                if embedding:

                        for row in reader:

                            ## When using Embedding
                            sequences.append(row[0])
                            
                            train_dataset.append([row[0],[int(row[1])]])
                else:
                        for row in reader:
                                
                                train_dataset.append([seqtopad(row[0],self.motiflen),[int(row[1])]])
  
        random.shuffle(train_dataset)
        size=int(len(train_dataset)/3)
        firstvalid=train_dataset[:size]
        secondvalid=train_dataset[size:size+size]
        thirdvalid=train_dataset[size+size:]
        firsttrain=secondvalid+thirdvalid
        secondtrain=firstvalid+thirdvalid
        thirdtrain=firstvalid+secondvalid
        return firsttrain,firstvalid,secondtrain,secondvalid,thirdtrain,thirdvalid,train_dataset,sequences
        
def Gen_Words(sequences,kmer_len,s): # this function is used to generate kmer list from sequences
    out=[]

    for i in sequences:

        kmer_list=[]
        for j in range(0,(len(i)-kmer_len)+1,s):

              kmer_list.append(i[j:j+kmer_len])

        out.append(kmer_list)

    return out

class chipseq_dataset(Dataset): # this class is used to generate dataset for training
    """ Diabetes dataset."""

    def __init__(self,xy=None):
        self.x_data=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
      

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

class chipseq_dataset_embd(Dataset):  # this class is used to generate dataset for training with embedding
    """ Diabetes dataset."""

    def __init__(self,xy=None,model=None,kmer_len=5,stride=2):
      
        self.kmer_len= kmer_len
        self.stride= stride
        data=[el[0] for el in xy]
        words_doc= self.Gen_Words(data,self.kmer_len,self.stride)
#         print(words_doc[0])
        x_data=[self.convert_data_to_index(el,model.wv) for el in words_doc]
#         print(x_data[0])
       
        
        self.x_data=np.asarray(x_data,dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.LongTensor(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
      

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
      
    def Gen_Words(self,pos_data,kmer_len,s):
        out=[]
        
        for i in pos_data:

            kmer_list=[]
            for j in range(0,(len(i)-kmer_len)+1,s):

                  kmer_list.append(i[j:j+kmer_len])
                
            out.append(kmer_list)
            
        return out


    def convert_data_to_index(self, string_data, wv): # this function is used to convert kmer list to index list
      index_data = []
      for word in string_data:
          if word in wv:
              index_data.append(wv.vocab[word].index)
      return index_data
      
      
class Chip_test(): # this class is used to generate test dataset
    def __init__(self,filename,motiflen=24):
        self.file = filename
        self.motiflen = motiflen
    def openFile(self):
        test_dataset=[]
        seq=[]


        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')
            if embedding:
                if evaluate_performance: 
                    for row in reader:
						## When using Embedding
                        test_dataset.append([row[0],[int(row[1])]])
                        seq.append(row[0])
                else:
                    for row in reader:
						## just adding fake label but it will not be used
                        test_dataset.append([row[0],[1]])
                        seq.append(row[0])
						
            else:
                if evaluate_performance: 
                    for row in reader:
                        test_dataset.append([seqtopad(row[0],self.motiflen),[int(row[1])]])
                        seq.append(row[0])
                else:
                    for row in reader:
						## just adding fake label but it will not be used
                        test_dataset.append([seqtopad(row[0],self.motiflen),[1]])
                        seq.append(row[0])
        return test_dataset,seq    



train_dataloader=[]
valid_dataloader=[]
test_loader=[]
sequences=[]
seq=[]

In [44]:
# torch.randn(nummotif,self.input_channels,motiflen)
torch.randn(1,4,6)

tensor([[[-9.2913e-01,  2.7619e-01, -5.3888e-01,  4.6258e-01, -8.7189e-01,
          -2.7118e-02],
         [-3.5325e-01,  1.4639e+00,  5.6351e-01,  1.8582e+00,  1.0441e+00,
          -8.6382e-01],
         [ 8.3509e-01, -3.1571e-01,  2.6911e-01,  8.5404e-02, -1.4129e+00,
          -1.8791e+00],
         [-1.7983e-01,  7.9039e-01,  5.2394e-01, -2.6935e-01, -1.6191e+00,
           1.2588e-03]]])

In [ ]:
class Network(nn.Module):
    def __init__(self, nummotif,motiflen,RNN_hidden_size,hidden_size,hidden,dropprob,sigmaConv,sigmaNeu,sigmaRNN,xavier_init):
      
        super(Network, self).__init__()
        
        self.hidden=hidden
        self.RNN_hidden_size=RNN_hidden_size
        
        self.dropprob=dropprob
        self.sigmaConv=sigmaConv
        self.sigmaNeu=sigmaNeu
        self.hidden_size=hidden_size  
        self.input_channels=4
        
        # Embedding
        if embedding:
              model1 = gensim.models.Word2Vec.load(word2vec_model)
              weights = torch.FloatTensor(model1.wv.vectors)
              self.embedding = nn.Embedding.from_pretrained(weights, freeze=False)
              self.input_channels=Embsize
              
        # Convnet
        self.ConvWeights=[]
        self.ConvBias=[]
        self.wConv=torch.randn(nummotif,self.input_channels,motiflen).to(device)
        self.wRect=torch.randn(nummotif).to(device)
        self.ConvWeights.append(self.wConv)
        self.ConvBias.append(self.wRect)
        conv_channels=nummotif
        
        if conv:
              self.FC_size= nummotif
              self.input_channels=nummotif
              for c in range(1,conv_layers):
                  Wconv=torch.randn(int(1.5*c*nummotif),conv_channels,motiflen).to(device)
                  Bconv=torch.randn(int(1.5*c*nummotif)).to(device)
                  self.ConvWeights.append(Wconv)
                  self.ConvBias.append(Bconv)
                  conv_channels=int(1.5*c*nummotif) # number of channels in the next layer
                  self.FC_size= int(1.5*c*nummotif)
                  self.input_channels=int(1.5*c*nummotif)
              torch.nn.init.normal_(self.ConvWeights[0],mean=0,std=sigmaConv)
              
              ind=0
              for weights in self.ConvWeights:
                  weights.requires_grad=True
                  if ind>0:
                    if dilation>1:
                      torch.nn.init.normal_(weights,mean=0,std=0.1)
                      print('ffffffff')
                    else:
                      torch.nn.init.xavier_uniform(weights)
                  ind=ind+1
              for weights in self.ConvBias:
                  weights.requires_grad=True
                  torch.nn.init.normal_(weights)
            
        
        # RNN
        self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
        if RNN:
              if RNN_type=='GRU':
                  self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=False).to(device)
                  self.FC_size= RNN_hidden_size
              elif RNN_type=='BiGRU':
                  self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=True).to(device)
                  self.FC_size= 2*RNN_hidden_size
              elif RNN_type=='LSTM':
                  self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=False).to(device)
                  self.FC_size= RNN_hidden_size
              elif RNN_type=='BiLSTM':
                  self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=True).to(device)
                  self.FC_size= 2*RNN_hidden_size
              if not xavier_init:
                for layer_p in self.rnn._all_weights:
                  for p in layer_p:
                    if 'weight' in p:
                      torch.nn.init.normal_(self.rnn.__getattr__(p),mean=0,std=sigmaRNN)
              else:
                for layer_p in self.rnn._all_weights:
                  for p in layer_p:
                    if 'weight' in p:
                      torch.nn.init.xavier_uniform(self.rnn.__getattr__(p))

        # FC    
        self.wHidden=torch.randn(self.FC_size,self.hidden_size).to(device)
        self.wHiddenBias=torch.randn(self.hidden_size).to(device)
        self.wHidden.requires_grad=True
        self.wHiddenBias.requires_grad=True
        
        if not self.hidden:
            self.wNeu=torch.randn(self.FC_size,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            if not xavier_init:
              torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
            else:
              torch.nn.init.xavier_uniform(self.wNeu)
            
            

        else:
           
            self.wNeu=torch.randn(self.hidden_size,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)     
            if not xavier_init:  
              torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wHidden,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wHiddenBias,mean=0,std=self.sigmaNeu)
            else:
              torch.nn.init.xavier_uniform(self.wNeu)
              torch.nn.init.xavier_uniform(self.wHidden)
            
  
           

        self.wNeu.requires_grad=True
        self.wNeuBias.requires_grad=True
        
        self.dropout=torch.nn.Dropout(p=dropprob, inplace=False)
        self.max=torch.nn.MaxPool1d(3, stride=1)
        
        
    def get_weights(self):
        ll=[]
        for layer_p in self.rnn._all_weights:
          for p in layer_p:
            if 'weight' in p:
               ll.append(self.rnn.__getattr__(p))
        return ll+self.ConvWeights+self.ConvBias
        
    def layer1out(self,x):
		
        if type(x) is np.ndarray:
          x = torch.from_numpy(x.astype(np.float32))
        #x = Variable(x, volatile=True)
        if torch.cuda.is_available():
          x = x.to(device)
        if embedding:
          print(x.shape)
          x= self.embedding(x)
          x=x.permute(0,2,1)
          print(x.shape)
        if conv:
          x=F.conv1d(x, self.wConv, bias=self.wRect, stride=1, padding=0)
          out=x.clamp(min=0)
          print(out.shape)
          temp = out.data.cpu().numpy()
        else:
          print('you need to have CNN to visualize motifs')
        return temp
        
    def forward(self, x):
      
        if embedding:
          # shape of x : batch_size x seq_len
          x= self.embedding(x)
          x=x.permute(0,2,1)
          # shape of x_emb : batch_size x embd_size x seq_len
          
#         else:
#           # shape of x : batch_size x 4 x seq_len
         
        if conv:
          x=F.conv1d(x, self.ConvWeights[0], bias=self.ConvBias[0], stride=1, padding=0)
          x=x.clamp(min=0)
          x=self.max(x)
          
          for c in range(1,len(self.ConvWeights)):
            x=F.conv1d(x, self.ConvWeights[c], bias=self.ConvBias[c], stride=1, padding=0,dilation=dilation)
            x=x.clamp(min=0)
            x=self.max(x)
            
        if RNN:
          if conv:
            
            x=self.dropout(x)
          x=x.permute(2,0,1)
          # shape of x :  seq_len x batch_size  x features
          output, _ = self.rnn(x)
          # shape of output :  seq_len x batch_size  x num_directions * features
          
          if RNN_type== 'BiLSTM' or RNN_type=='BiGRU':

              Normal_RNN=output[-1, :, :self.RNN_hidden_size]
              Rev_RNN=output[0, :, self.RNN_hidden_size:]
              x = torch.cat((Normal_RNN, Rev_RNN), 1)
              x=self.dropout(x)
              #shape of x: batch_size x 2*hidden_size
#               print(x.shape)
              
          else:
                      ## from (1, N, hidden) to (N, hidden)
              x = output[-1, :, :]
              x=self.dropout(x)
#               print(hn.shape)
#               x = hn.view(hn.size()[1], hn.size(2))
              # shape of x: batch_size x hidden_size
              #print(x.shape)
          
          
        else:
          x, _ = torch.max(x, dim=2)
          #print(x.shape)

          # shape of x : batch_size x numb_filters
          x=self.dropout(x)
        if self.hidden:
          x=x @ self.wHidden
          x.add_(self.wHiddenBias)
          x=x.clamp(min=0)
          x=self.dropout(x)
        x=x @ self.wNeu
        x.add_(self.wNeuBias)
          
        
        
        return torch.sigmoid(x)

In [39]:
stride = 1 # stride of sliding window
Embepochs = 100 # number of iterations
Embsize = 50 # embedding size
print('training word2vec model')
document= Gen_Words(train_data['acpM-kasA'], 25, 1)
model = gensim.models.Word2Vec (document, window=int(12 / stride), min_count=0, vector_size=Embsize, workers=multiprocessing.cpu_count())
model.train(document,total_examples=len(document),epochs=Embepochs)
model.save('word2vec_model')

training word2vec model


In [ ]:
def Calibration():
	print('start')
	best_AUC=0
	device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
	print(device)
	# device='cpu'
	learning_steps_list=[5000,10000,15000,20000,25000,30000,35000,40000]
	for number in range(40):
		# hyper-parameters
		RNN_hidden_size_list=[20,50,80,100]
		RNN_hidden_size=random.choice(RNN_hidden_size_list)
		dropoutList=[0,0.15,0.3,0.45,0.6] 
		dropprob=random.choice(dropoutList)
		hidden_list=[True,False]
		hidden=random.choice(hidden_list)
		xavier_List=[True,True,False] 
		xavier=random.choice(xavier_List)
		hidden_size_list=[32,64]
		hidden_size=random.choice(hidden_size_list)
		optim_list=['SGD','Adagrad','Adagrad']
		optim=random.choice(optim_list)
		learning_rate=logsampler(0.005,0.5) 
		momentum_rate=sqrtsampler(0.95,0.99)  
		sigmaConv=logsampler(10**-6,10**-2)   
		sigmaNeu=logsampler(10**-3,10**-1) 
		sigmaRNN=logsampler(10**-4,10**-1) 
		weightDecay=logsampler(10**-10,10**-1) 
		nummotif_list=[16]
		nummotif1=random.choice(nummotif_list)
		
		
		
	
		
		model_auc=[[],[],[]]
		for kk in range(3):
			model = Network(nummotif1,motiflen,RNN_hidden_size,hidden_size,hidden,dropprob,sigmaConv,sigmaNeu,sigmaRNN,xavier).to(device)
			if optim=='SGD':
				optimizer = torch.optim.SGD(model.get_weights()+[model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=learning_rate,momentum=momentum_rate,nesterov=True
											,weight_decay=weightDecay)
			else:
				optimizer = torch.optim.Adagrad(model.get_weights()+[model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=learning_rate,weight_decay=weightDecay)

			train_loader=train_dataloader[kk]
			valid_loader=valid_dataloader[kk]
			
			
			learning_steps=0
			while learning_steps<=40000:
			   
				auc=[]
				model.train()
				for i, (data, target) in enumerate(train_loader):
					
					data = data.to(device)
					target = target.to(device)

					# Forward pass
					output = model(data)          
					loss = F.binary_cross_entropy(output,target)
					optimizer.zero_grad()
					loss.backward()
					optimizer.step()
					learning_steps+=1

					if learning_steps% 5000==0:
						
							
							
							
						with torch.no_grad():
							model.eval()
							auc=[]
							for j, (data1, target1) in enumerate(valid_loader):
								data1 = data1.to(device)
								target1 = target1.to(device)
								
								# Forward pass
								output = model(data1)
								
								pred=output.cpu().detach().numpy().reshape(output.shape[0])
								labels=target1.cpu().numpy().reshape(output.shape[0])
								if output.shape[0]>60:
									auc.append(metrics.roc_auc_score(labels, pred))
							#print(np.mean(auc))
							model_auc[kk].append(np.mean(auc))
							
							model.train()
						
		
		print('                   ##########################################               ')

		for n in range(8):
			AUC=(model_auc[0][n]+model_auc[1][n]+model_auc[2][n])/3
			#print(AUC)
			if AUC>best_AUC:
				best_AUC=AUC
				best_learning_steps=learning_steps_list[n]
				best_LearningRate=learning_rate
				best_LearningMomentum=momentum_rate
				best_sigmaConv=sigmaConv
				best_dropprob=dropprob
				best_sigmaNeu=sigmaNeu
				best_RNN_hidden_size=RNN_hidden_size
				best_weightDecay=weightDecay
				best_hidden=hidden
				best_sigmaRNN=sigmaRNN
				best_xavier=xavier
				best_optim=optim
				best_nummotif=nummotif
				best_hidden_size=hidden_size


	print('best_AUC=',best_AUC)            
	print('best_learning_steps=',best_learning_steps)      
	print('best_LearningRate=',best_LearningRate)
	print('best_LearningMomentum=',best_LearningMomentum)
	print('best_sigmaConv=',best_sigmaConv)
	print('best_dropprob=',best_dropprob)
	print('best_sigmaNeu=',best_sigmaNeu)
	print('best_RNN_hidden_size',best_RNN_hidden_size)
	print('best_weightDecay=',weightDecay)
	print('best_hidden=',best_hidden)
	print('best_sigmaRNN=',best_sigmaRNN)
	print('best_xavier=',best_xavier)
	print('best_optim=',best_optim)
	print('best_nummotif=',best_nummotif)
	print('best_hidden_size=',best_hidden_size)
	
	best_hyperparameters = {'best_learning_steps': best_learning_steps,'best_LearningRate':best_LearningRate,'best_LearningMomentum':best_LearningMomentum,'best_sigmaConv':best_sigmaConv,
	                         'best_dropprob':best_dropprob,'best_sigmaNeu':best_sigmaNeu,'best_RNN_hidden_size':best_RNN_hidden_size,
	                         'best_weightDecay':best_weightDecay,'best_hidden':best_hidden,'best_sigmaRNN':best_sigmaRNN,'best_xavier':best_xavier,'best_optim':best_optim,'best_nummotif':best_nummotif,'best_hidden_size':best_hidden_size}
	torch.save(best_hyperparameters, model_dir+'best_hyperpamarameters.pth')
	return best_hyperparameters
def Train_model():
	best_hyperparameters=torch.load(model_dir+'best_hyperpamarameters.pth')
	best_learning_steps=best_hyperparameters['best_learning_steps']
	best_LearningRate=best_hyperparameters['best_LearningRate']
	best_LearningMomentum=best_hyperparameters['best_LearningMomentum']
	best_sigmaConv=best_hyperparameters['best_sigmaConv']
	best_dropprob=best_hyperparameters['best_dropprob']
	best_sigmaNeu=best_hyperparameters['best_sigmaNeu']
	best_RNN_hidden_size=best_hyperparameters['best_RNN_hidden_size']
	best_weightDecay=best_hyperparameters['best_weightDecay']
	best_hidden=best_hyperparameters['best_hidden']
	best_sigmaRNN=best_hyperparameters['best_sigmaRNN']
	best_xavier=best_hyperparameters['best_xavier']
	best_optim=best_hyperparameters['best_optim']
	best_nummotif=best_hyperparameters['best_nummotif']
	best_hidden_size=best_hyperparameters['best_hidden_size']
	best_AUC=0


	for number_models in range(5):
	  model = Network(best_nummotif,motiflen,best_RNN_hidden_size,best_hidden_size,best_hidden,best_dropprob,best_sigmaConv,best_sigmaNeu,best_sigmaRNN,best_xavier).to(device)
	  if best_optim=='SGD':
		  optimizer = torch.optim.SGD(model.get_weights()+[model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=best_LearningRate,momentum=best_LearningMomentum,nesterov=True,weight_decay=best_weightDecay)
	  else:
		  optimizer = torch.optim.Adagrad(model.get_weights()+[model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=best_LearningRate,weight_decay=best_weightDecay)


	  train_loader=alldataset_loader
	  valid_loader=alldataset_loader
	  learning_steps=0
	  model.train()
	  while learning_steps<=best_learning_steps:
	  
		  for i, (data, target) in enumerate(train_loader):
			  data = data.to(device)
			  target = target.to(device)
			  
				# Forward pass
			  output = model(data)
			  loss = F.binary_cross_entropy(output,target)

			  optimizer.zero_grad()
			  loss.backward()
			  optimizer.step()
			  learning_steps+=1
			  
	  with torch.no_grad():
		  model.eval()
		  auc=[]
		  for i, (data, target) in enumerate(valid_loader):
			  data = data.to(device)
			  target = target.to(device)
			  
			  # Forward pass
			  output = model(data)
			  
			  pred=output.cpu().detach().numpy().reshape(output.shape[0])
			  labels=target.cpu().numpy().reshape(output.shape[0])
			  if output.shape[0]>30:
				  auc.append(metrics.roc_auc_score(labels, pred))
	  #             
		  AUC_training=np.mean(auc)
		  print('AUC on training data for model ',number_models+1,' = ',AUC_training)
		  if AUC_training>best_AUC:
			  best_AUC=AUC_training
			  best_model=model
	torch.save(best_model, model_path)
	#torch.save(best_model.state_dict(), model_dir+'best_model.pkl')
	return best_model
#### save model .pkl
#### load model